The real coordinates of the data from the SIDER2 in the chromosome 1 is:
1) plus 24099-24758 !
2) minus 35933-35313 !
3) minus 55758-54697 !
4) minus 114223-113357 ! 
5) minus 130738-129878 ! 
6) plus 137266-138019 ! 
7) plus 205359-206213 !

Lets make a fasta file with the csv data from "./RUN20_LinJ.01.csv"

In [80]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [81]:
linj01_df = pd.read_csv("./RUN20_LinJ.01.csv", sep=",", header=None)

In [82]:
def fasta_creator(path_input, fasta_output_path):

    data_df = pd.read_csv(path_input, sep=",", header=None)

    fasta_df = []
    for index, row in enumerate(data_df.iterrows()):
        row = row[1]
        rec = SeqRecord(
            Seq(row.iloc[5]),
            id="Seq_" + str(index) + "_" + row.iloc[0] + "_" + row.iloc[4],  # Que tenga aqui el sentido es esencial para luego filtrarlos
            description="Leishmania infantum " + row.iloc[4]
        )
        fasta_df.append(rec)

    SeqIO.write(fasta_df, fasta_output_path, "fasta")
    print("\nFasta created at:", fasta_output_path)

In [83]:
fasta_creator("RUN20_LinJ.01.csv", "RUN20_LinJ.01.fasta")


Fasta created at: RUN20_LinJ.01.fasta


# Create a data frame to compare

In [84]:
data =[
        ["plus", "minus", "minus", "minus", "minus", "plus", "plus"],
        [24099, 35933, 55758, 114223, 130738, 137266, 205359],
        [24758, 35313, 54697, 113357, 129878, 138019, 206213],
        [24093, 36163, 55775, 114388, 130854, 137458, 205558],
        [24758, 35374, 54986, 113760, 130227, 137919, 206003]
    ]

data_transposed = list(zip(*data))

compare_df = pd.DataFrame(data_transposed,
    columns=["strand", "Start_TP", "End_TP", "Start_Test", "End_Test"]
)

compare_df.head()

,strand,Start_TP,End_TP,Start_Test,End_Test
0,plus,24099,24758,24093,24758
1,minus,35933,35313,36163,35374
2,minus,55758,54697,55775,54986
3,minus,114223,113357,114388,113760
4,minus,130738,129878,130854,130227


In [85]:
compare_df["Start_Difference"] = compare_df.apply(lambda x: x["Start_Test"] - x["Start_TP"], axis=1)
compare_df["End_Difference"] = compare_df.apply(lambda x: x["End_Test"] - x["End_TP"], axis=1)
compare_df.head()


,strand,Start_TP,End_TP,Start_Test,End_Test,Start_Difference,End_Difference
0,plus,24099,24758,24093,24758,-6,0
1,minus,35933,35313,36163,35374,230,61
2,minus,55758,54697,55775,54986,17,289
3,minus,114223,113357,114388,113760,165,403
4,minus,130738,129878,130854,130227,116,349


In [86]:
seq_len_TP = []
for value in compare_df.iterrows():
    value = value[1]
    seq_len_TP.append(abs(value["End_TP"] - value["Start_TP"]) + 1)
    # print(abs(value.loc["End_TP"] - value.loc["Start_TP"]) + 1)

seq_len_Test = []
for value in compare_df.iterrows():
    value = value[1]
    seq_len_Test.append(abs(value["End_Test"] - value["Start_Test"]) + 1)
    # print(abs(value.loc["End_Test"] - value.loc["Start_Test"]) + 1)

In [87]:
compare_df["Seq_Len_TP"] = seq_len_TP
compare_df["Seq_Len_Test"] = seq_len_Test
compare_df["Seq_Len_Difference"] = compare_df["Seq_Len_Test"] - compare_df["Seq_Len_TP"]

In [115]:
compare_df

,strand,Start_TP,End_TP,Start_Test,End_Test,Start_Difference,End_Difference,Seq_Len_TP,Seq_Len_Test,Seq_Len_Difference
0,plus,24099,24758,24093,24758,-6,0,660,666,6
1,minus,35933,35313,36163,35374,230,61,621,790,169
2,minus,55758,54697,55775,54986,17,289,1062,790,-272
3,minus,114223,113357,114388,113760,165,403,867,629,-238
4,minus,130738,129878,130854,130227,116,349,861,628,-233
5,plus,137266,138019,137458,137919,192,-100,754,462,-292
6,plus,205359,206213,205558,206003,199,-210,855,446,-409


In [91]:
list(compare_df["Start_Test"])

[24093, 36163, 55775, 114388, 130854, 137458, 205558]

In [96]:
plus_list = list(compare_df[compare_df["strand"] == "plus"]["Start_Test"])
minus_list = list(compare_df[compare_df["strand"] == "minus"]["Start_Test"])

counter = 1
for value in pd.read_csv("../Data/To_analyze/RUNS/run_19.csv", sep=",", header=None).iterrows():
    value = value[1]
    if value[1] == "LinJ.01" and value[14] == "plus":
        if value[10] in plus_list:
            print(f"{counter} sequence in PLUS found START in {value[10]}")
            counter += 1
    
    elif value[1] == "LinJ.01" and value[14] == "minus":
        if value[10] in minus_list:
            print(f"{counter} sequence in MINUS found START in {value[10]}")
            counter += 1

    

1 sequence in PLUS found START in 24093
2 sequence in PLUS found START in 137458
3 sequence in PLUS found START in 205558
4 sequence in MINUS found START in 36163
5 sequence in MINUS found START in 55775
6 sequence in MINUS found START in 114388
7 sequence in MINUS found START in 130854


In [123]:
plus_list

[[24093, 137458, 205558], [24758, 138019, 206213]]

In [125]:
minus_list

[[36163, 55775, 114388, 130854], [35313, 54697, 113357, 129878]]

In [150]:
plus_list = [list(compare_df[compare_df["strand"] == "plus"]["Start_Test"]),
            list(compare_df[compare_df["strand"] == "plus"]["End_TP"])
]
minus_list = [list(compare_df[compare_df["strand"] == "minus"]["Start_Test"]),
            list(compare_df[compare_df["strand"] == "minus"]["End_TP"])
]

In [152]:
minus_list

[[36163, 55775, 114388, 130854], [35313, 54697, 113357, 129878]]

In [149]:
def comparative_csv(path_input, coor, plus_list, minus_list):
    counter = 1
    for value in pd.read_csv(path_input, sep=",", header=None).iterrows():
        value = value[1]

        if coor == "start": 
            coor2 = value[10]
            plus_list = plus_list[0]
            minus_list = minus_list[0]
        elif coor == "end": 
            coor2 = value[11]
            plus_list = plus_list[1]
            minus_list = minus_list[1]

        if value[1] == "LinJ.01" and value[14] == "plus":
            print(coor2)
            print(type(coor2))
            print(plus_list)
            if coor2 in plus_list:
                print(f"{counter} sequence in PLUS found {coor.upper()} in {coor2}")
                counter += 1
        
        elif value[1] == "LinJ.01" and value[14] == "minus":
            if coor2 in minus_list:
                print(f"{counter} sequence in MINUS found {coor.upper()} in {coor2}")
                counter += 1
comparative_csv("../Data/To_analyze/RUNS/run_19.csv", "start", plus_list, minus_list)

24093
<class 'int'>
[24093, 137458, 205558]
1 sequence in PLUS found START in 24093
35316
<class 'int'>
24093


TypeError: argument of type 'int' is not iterable

In [143]:
comparative_csv("../Data/To_analyze/RUNS/run_19.csv", "start", plus_list, minus_list)

24093
<class 'int'>
1 sequence in PLUS found START in 24093
35316
<class 'int'>


TypeError: argument of type 'int' is not iterable